# Model Performance


## Prerequisites
Install python packages

In [16]:
%%capture
%pip install pandas pydicom pydicom-seg scikit-learn seaborn segmentationmetrics requests

In [17]:
import warnings

warnings.filterwarnings("ignore")

from pathlib import Path
import requests
import zipfile
import subprocess
import io
import numpy as np
import pandas as pd
import scipy.stats as stats
import pydicom
import pydicom_seg
import SimpleITK as sitk
import matplotlib.pyplot as plt
import segmentationmetrics as sm
from segmentationmetrics.surface_distance import compute_surface_dice_at_tolerance
import seaborn as sns
from tqdm.auto import tqdm

sns.set_theme(style="whitegrid")
%matplotlib inline


## Download the segmentation results from zenodo

In [18]:
qa_dir = Path("qa-results")

In [19]:
def download_inference(
    extract_dir="qa-results", record="12734644", filename="lung2-ct.zip"
):
    url = (f"https://zenodo.org/record/{record}/files/{filename}")
    # download the zip file and extract it
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(extract_dir)


if not qa_dir.exists() or not (qa_dir / "ai-segmentations-dcm").exists():
    download_inference(qa_dir)

In [20]:
plt_colors = {
    "ne2": "#5eceb0",
    "rad1": "#9e70e1",
    "tp": "k",
    "fp": "b",
    "fn": "r",
}

revewer_cmap = sns.color_palette([plt_colors["ne2"], plt_colors["rad1"]])
vol_cmap = sns.color_palette([plt_colors["fp"], plt_colors["fn"]])

In [21]:
def load_dcm_seg(seg_path: Path, label=1):
    reader = pydicom_seg.SegmentReader()
    dcm = pydicom.dcmread(str(seg_path))
    seg = reader.read(dcm)
    # unique_labels = list(seg._segment_data.keys())
    # print(unique_labels)
    if label in seg._segment_data:
        result = sitk.GetImageFromArray(seg._segment_data[label])
    else:
        print(
            f"Label {label} not found in the DICOM segmentation file. Returning a zero-valued image."
        )
        # Create a zero-valued image with the same dimensions, origin, spacing, and direction
        zero_array = np.zeros_like(seg._segment_data[next(iter(seg._segment_data))])
        result = sitk.GetImageFromArray(zero_array)
    # result = sitk.Cast(result, sitk.sitkUInt8)
    # result.SetOrigin(seg.origin)
    # result.SetSpacing(seg.spacing)
    # result.SetDirection(seg.direction)
    result = sitk.Cast(result, sitk.sitkUInt8)
    return result


def load_nii_seg(seg_path: Path, label=1):
    img = sitk.ReadImage(str(seg_path))
    if label==1:
        img = sitk.Cast(img >= label, sitk.sitkUInt8)
    else:
        img = sitk.Cast(img == label, sitk.sitkUInt8)
    return img


def load_seg(seg_path: Path, label=1):
    if seg_path.suffix == ".dcm":
        return load_dcm_seg(seg_path, label)
    else:
        return load_nii_seg(seg_path, label)


def resize_label(img: sitk.Image, ref_img: sitk.Image, interp=sitk.sitkNearestNeighbor):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_img)
    resampler.SetInterpolator(interp)
    resampler.SetDefaultPixelValue(0)
    resampled_img = resampler.Execute(img)
    return resampled_img


def calc_metrics_for_label(ai_seg_file, qa_seg_file, fname, label_value=1, version="aimiv2", label_suffix=""):
    ai_img = load_seg(ai_seg_file, label_value)
    qa_img = load_seg(qa_seg_file, label_value)
    qa_img = resize_label(qa_img, ai_img)  # match the size of the ai_img
    ai_arr = sitk.GetArrayFromImage(ai_img)
    qa_arr = sitk.GetArrayFromImage(qa_img)
    spacing = ai_img.GetSpacing()[::-1]  # numpy is reversed dimensions from sitk

    qa_img = resize_label(qa_img, ai_img)  # match the size of the ai_img
    m = sm.SegmentationMetrics(ai_arr, qa_arr, spacing)

    metrics = {
        "dice": m.dice,
        "hausdorff_distance_95": m.hausdorff_distance,
        "mean_surface_distance": m.mean_surface_distance,
        "mean_surface_distance_tol_7": compute_surface_dice_at_tolerance(
            m._surface_dist, 7
        ),
        "SeriesInstanceUID": fname,  # from medical segmentation decathlon
        "label": label_value,
        "version": version,
    }
    print(metrics)
    if label_suffix:
        metrics = {f"{k}_{label_suffix}": v for k, v in metrics.items()}
    return metrics

In [22]:
from pathlib import Path
import pandas as pd
import os
from tqdm.auto import tqdm
import subprocess

aimiv1_metrics, aimiv2_metrics = [], []
# Load the CSV file
df = pd.read_csv(Path("qa-results/qa-results.csv").resolve())

flist = []
aimiv1 = Path("qa-results/aimiv1-ai-segmentations").resolve()
aimiv2 = Path("qa-results/ai-segmentations-dcm").resolve()
# qa-list
qapath = Path("qa-results/qa-segmentations-dcm").resolve()
idx = 0
for file in qapath.glob("*dcm"):
    # if idx==10:
    #     continue
    fname = Path(file.stem).stem.split("qa-")[-1]
    aimiv1_fname = aimiv1 / f"aimiv1-{fname}.nii.gz"
    aimiv2_fname = aimiv2 / f"ai-{fname}.seg.dcm"
    qa_seg_file = qapath / f"qa-{fname}.seg.dcm"
    assert aimiv1_fname.exists()
    assert aimiv2_fname.exists()
    assert qa_seg_file.exists()
    # aimiv1_metrics.append(
    #     calc_metrics_for_label(
    #         aimiv1_fname, qa_seg_file, fname, version="aimiv1", label_value=2
    #     )
    # )
    # aimiv1_metrics.append(
    #     calc_metrics_for_label(
    #         aimiv1_fname, qa_seg_file, fname, version="aimiv1", label_value=1
    #     )
    # )
    aimiv2_metrics.append(
        calc_metrics_for_label(aimiv2_fname, qa_seg_file, fname, label_value=2)
    )
    aimiv2_metrics.append(
        calc_metrics_for_label(aimiv2_fname, qa_seg_file, fname, label_value=1)
    )
    idx += 1

aimiv1_df = pd.DataFrame(aimiv1_metrics)
aimiv2_df = pd.DataFrame(aimiv2_metrics)
# # Prefix columns
# aimiv1_df = aimiv1_df.add_prefix("aimiv1_")
# aimiv1_df = aimiv1_df.rename(columns={"aimiv1_SeriesUID": "SeriesUID"})
# aimiv2_df = aimiv2_df.add_prefix("aimiv2_")
# aimiv2_df = aimiv2_df.rename(columns={"aimiv2_SeriesUID": "SeriesUID"})

# # Merge DataFrames on 'subjects'
# combined_df = pd.merge(aimiv1_df, aimiv2_df, on="SeriesUID")

# # Replace infinite values with NaN and drop NaN values
# combined_df = combined_df.replace([np.inf, -np.inf], np.nan).dropna()

# # Combine both aimiv1 and aimiv2 DataFrames into a single DataFrame for aggregation
# aimiv1_cols = [
#     "SeriesUID",
#     "aimiv1_label",
#     "aimiv1_dice",
#     "aimiv1_hausdorff_distance_95",
#     "aimiv1_mean_surface_distance",
#     "aimiv1_mean_surface_distance_tol_7",
#     "aimiv1_version",
# ]
# aimiv2_cols = [
#     "SeriesUID",
#     "aimiv2_label",
#     "aimiv2_dice",
#     "aimiv2_hausdorff_distance_95",
#     "aimiv2_mean_surface_distance",
#     "aimiv2_mean_surface_distance_tol_7",
#     "aimiv2_version",
# ]

# # Rename columns for easier aggregation
# df1 = combined_df[aimiv1_cols].rename(columns=lambda x: x.replace("aimiv1_", ""))
# df2 = combined_df[aimiv2_cols].rename(columns=lambda x: x.replace("aimiv2_", ""))

# # Concatenate aimiv1 and aimiv2 DataFrames for aggregation
# combined_df = pd.concat([df1, df2])

# # Group by 'label' and 'Reviewer' and aggregate the specified columns
# aggregated_df = (
#     combined_df.groupby(["label", "version"])
#     .agg(
#         {
#             "dice": ["mean", "std"],
#             "hausdorff_distance_95": ["mean", "std"],
#             "mean_surface_distance": ["mean", "std"],
#             "mean_surface_distance_tol_7": ["mean", "std"],
#         }
#     )
#     .round(2)
# )

{'dice': 0.0, 'hausdorff_distance_95': 101.06433594498111, 'mean_surface_distance': 78.97215191680671, 'mean_surface_distance_tol_7': 0.0, 'SeriesInstanceUID': '1.3.283555618156207794581815676543265885365354236113', 'label': 2, 'version': 'aimiv2'}


KeyboardInterrupt: 

In [ ]:
# Separate data for label1 (lung) and label2 (lesion) for aimiv2
aimiv2_label1_df = aimiv2_df[aimiv2_df["aimiv2_label"] == 1]
aimiv2_label2_df = aimiv2_df[aimiv2_df["aimiv2_label"] == 2]

# Create a figure and axis for subplots
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))

# List of metrics to plot
metrics = [
    "aimiv2_dice",
    "aimiv2_hausdorff_distance_95",
    "aimiv2_mean_surface_distance",
    "aimiv2_mean_surface_distance_tol_7",
]

# Plot each metric
for i, metric in enumerate(metrics):
    row = i // 2
    col = i % 2
    ax = axs[row, col]

    # Extract data for current metric
    label1_data = aimiv2_label1_df[metric]
    label2_data = aimiv2_label2_df[metric]

    # Check if label2_data has values and is not empty
    if not label2_data.empty and not label1_data.empty:
        # Convert data to numeric if necessary
        label1_data = pd.to_numeric(label1_data, errors="coerce")
        label2_data = pd.to_numeric(label2_data, errors="coerce")

        # Drop NaN values if any (optional)
        label1_data = label1_data.dropna()
        label2_data = label2_data.dropna()

        # Check if after dropping NaN, there's still data
        if not label1_data.empty and not label2_data.empty:
            # Create boxplots side by side
            ax.boxplot([label1_data, label2_data], patch_artist=True, showmeans=True)
            ax.set_title(f"{metric.split('aimiv2_')[-1]}")
            ax.set_xticklabels(
                ["Lungs", "Lesions"]
            )  # Update x-axis labels to "Lesions"
        else:
            # Handle case where data might be dropped to empty
            print(f"No valid data available for {metric} after cleaning.")
    else:
        # If label2_data is empty, handle accordingly (e.g., print a message)
        print(f"No data available for {metric} in label2 (Lesions).")

# Adjust layout
plt.tight_layout()
plt.show()

NameError: name 'aimiv2_df' is not defined